In [2]:
import pandas as pd
import pyodbc
from datamart import fix_me
import useful_functions as use

In [3]:
username = 'vigrose'
password_edw = 'Hufflepuff~10946'
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)

In [12]:
ppd = use.get_ppd()

INFO:useful_functions:PPD file from Jan 08 2022
C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (13,14,20,21,49,56,57,72) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [3]:
#Get polos
sql_query_polo = \
    '''
    SELECT
    A.PARTY_ID,
    A.POST_CD_ID,
    A.CAT_CD_ID,
    P.ADDR_1,
    P.ADDR_2,
    P.ADDR_3,
    P.CITY,
    S.SRC_STATE_CD AS STATE,
    P.POST_CD AS ZIP,
    A.FROM_DT,
    P.SRC_POST_KEY AS COMM_ID,
    c.PURPOSE_TYPE_DESC
    FROM
    AMAEDW.POST_CD P, AMAEDW.STATE S , AMAEDW.PARTY_ADDR A, AMAEDW.CONT_PURPOSE_TYPE C
    WHERE
    A.PURPOSE_TYPE_ID=C.PURPOSE_TYPE_ID
    AND
    C.PURPOSE_CAT_CD='A'
    AND
    C.PURPOSE_USG_CD='PO'
    AND
    A.POST_CD_ID = P.POST_CD_ID
    AND
    P.STATE_ID = S.STATE_ID
    AND
    A.THRU_DT IS NULL;
    '''
POLOS = pd.read_sql(con=AMAEDW, sql=sql_query_polo)
POLOS.head()

,PARTY_ID,POST_CD_ID,CAT_CD_ID,ADDR_1,ADDR_2,ADDR_3,CITY,STATE,ZIP,FROM_DT,COMM_ID,PURPOSE_TYPE_DESC
0,1849882,16269589,NaN,2603 University Blvd,,,Tuscaloosa,AL,35401,2005-12-16 20:20:34.850978,713677,Office
1,1843314,16267642,NaN,633 S Union Ave,,,Ozark,AL,36360,2005-12-16 20:20:34.853611,713743,Office
2,1849885,13850597,NaN,9135 Piscataway Rd Ste 235,,,Clinton,MD,20735,2005-12-16 20:20:34.854390,713765,Office
3,1888588,12193084,NaN,1106 Druid Rd S,,,Clearwater,FL,33756,2005-12-16 20:20:34.858320,713865,Office
4,1798055,16183293,NaN,106 Medical Arts Bldg,,,Anniston,AL,36201,2005-12-16 20:20:34.860548,713910,Office


In [107]:
POLOS[POLOS.CAT_CD_ID.isnull()].to_csv('../../Data/DEA/NULL_CAT_CD_ID.csv', index=False)

In [42]:
first_test = POLOS[(POLOS.CAT_CD_ID==4159.0)&(POLOS.ME.isin(dpc.ME))].sample(100)

In [73]:
dea_sourced = POLOS[(POLOS.CAT_CD_ID==4159.0)&(POLOS.ME.isin(with_polos.ME))]

In [6]:
#Get party ids
ME_QUERY = \
    """
    SELECT DISTINCT
    P.PARTY_ID,
    P.KEY_VAL AS ME
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 18
    AND
    P.ACTIVE_IND = 'Y'
    """
party_ids = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

In [13]:
POLOS = pd.merge(party_ids,POLOS, on='PARTY_ID')

In [14]:
POLOS[POLOS.ME.isin(ppd.ME)]

,PARTY_ID,ME,POST_CD_ID,CAT_CD_ID,ADDR_1,ADDR_2,ADDR_3,CITY,STATE,ZIP,FROM_DT,COMM_ID,PURPOSE_TYPE_DESC
0,1889235,00102990919,16542167,6455.0,101 Sivley Rd SW,Huntsville Hosp Emer Dept,,Huntsville,AL,35801,2021-12-10 07:24:36.414934,9467389,Office
1,1904405,00102840976,32396320,4032.0,5397 Edgewater Dr,,,Ewa Beach,HI,96706,2021-07-16 07:36:09.691684,26014209,Office
3,1892959,00106930210,16561405,NaN,1215 7th St SE Ste 110,,,Decatur,AL,35601,2005-12-16 20:35:36.375262,9806586,Office
14,1903519,00102540336,16090329,NaN,840 Montclair Rd Ste 724,,,Birmingham,AL,35213,2005-12-16 20:20:35.396053,714512,Office
19,1888621,00102550404,25638874,NaN,P O Box 1460,230 Hospital Dr,E R Physician,Vernon,AL,35592,2015-03-19 21:11:52.503690,19912709,Office
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1148479,6614039,67205120049,15577846,3811.0,1137 2nd St SW,,,Mason City,IA,50401,2022-01-07 07:19:48.221887,7591378,Office
1148480,10008444,60501180184,34472504,3812.0,4401 Penn Ave,,,Pittsburgh,PA,15224,2022-01-07 07:19:48.657299,28153662,Office
1148481,10832756,91306070130,26831541,3922.0,1 Medical Center Dr Box 8003,,,Morgantown,WV,26506,2022-01-07 07:19:48.666461,20853820,Office
1148482,16580054,03575161625,34539346,3812.0,14 Sweetgum Ln,,,Staten Island,NY,10314,2022-01-07 07:19:48.872826,28210788,Office


In [20]:
with_polos = ppd.dropna(subset=['POLO_MAILING_LINE_2'])

In [22]:
len(with_polos)

991987

In [23]:
168410/991987

0.1697703699746065

In [26]:
dpc = with_polos[with_polos.TOP_CD==20]

In [28]:
152811/727987

0.20990896815465113

In [30]:
POLOS.drop_duplicates('ME')

,PARTY_ID,ME,POST_CD_ID,CAT_CD_ID,ADDR_1,ADDR_2,ADDR_3,CITY,STATE,ZIP,FROM_DT,COMM_ID,PURPOSE_TYPE_DESC
0,1889235,00102990919,16542167,6455.0,101 Sivley Rd SW,Huntsville Hosp Emer Dept,,Huntsville,AL,35801,2021-12-10 07:24:36.414934,9467389,Office
1,1904405,00102840976,32396320,4032.0,5397 Edgewater Dr,,,Ewa Beach,HI,96706,2021-07-16 07:36:09.691684,26014209,Office
2,1894036,00401530789,11103248,NaN,1406 College Dr,Mitchell Young M D,,Texarkana,TX,75503,2005-12-16 20:27:40.957626,6454455,Office
3,1892959,00106930210,16561405,NaN,1215 7th St SE Ste 110,,,Decatur,AL,35601,2005-12-16 20:35:36.375262,9806586,Office
4,1891637,00102520483,16445710,NaN,8993 Woodhaven Dr,,,Pinson,AL,35126,2005-12-16 20:29:20.843635,7549648,Office
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1148479,6614039,67205120049,15577846,3811.0,1137 2nd St SW,,,Mason City,IA,50401,2022-01-07 07:19:48.221887,7591378,Office
1148480,10008444,60501180184,34472504,3812.0,4401 Penn Ave,,,Pittsburgh,PA,15224,2022-01-07 07:19:48.657299,28153662,Office
1148481,10832756,91306070130,26831541,3922.0,1 Medical Center Dr Box 8003,,,Morgantown,WV,26506,2022-01-07 07:19:48.666461,20853820,Office
1148482,16580054,03575161625,34539346,3812.0,14 Sweetgum Ln,,,Staten Island,NY,10314,2022-01-07 07:19:48.872826,28210788,Office


In [ ]:
1148484

In [31]:
test = pd.merge(with_polos, POLOS, on='ME')

In [39]:
test[['POLO_MAILING_LINE_1', 'POLO_MAILING_LINE_2', 'POLO_CITY', 'POLO_STATE',
       'POLO_ZIP', 'ADDR_1', 'ADDR_2', 'ADDR_3',
       'CITY_y', 'STATE_y', 'ZIP_y', 'FROM_DT_y']]

,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP,ADDR_1,ADDR_2,ADDR_3,CITY_y,STATE_y,ZIP_y,FROM_DT_y
0,NaN,315 W HICKORY ST,SYLACAUGA,AL,35150,315 W Hickory St,,,Sylacauga,AL,35150,2021-12-24 07:15:01.554842
1,NaN,8101 BROWNLEIGH DR,RALEIGH,NC,27617,8101 Brownleigh Dr,,,Raleigh,NC,27617,2008-06-19 22:10:16.708828
2,MOUNTAIN AREA PEDIATRIC ASSOC,500 CENTREPARK DR,ASHEVILLE,NC,28805,500 Centrepark Dr,Mountain Area Pediatric Assoc,,Asheville,NC,28805,2008-11-20 22:28:35.223701
3,NaN,14502 W MEEKER BLVD,SUN CITY WEST,AZ,85375,14502 W Meeker Blvd,,,Sun City West,AZ,85375,2015-10-01 22:25:17.582194
4,NaN,1514 JEFFERSON HWY,NEW ORLEANS,LA,70121,1514 Jefferson Hwy,,,New Orleans,LA,70121,2021-12-10 07:24:31.572676
...,...,...,...,...,...,...,...,...,...,...,...,...
990722,HERITAGE VALLEY HEALTH SYSTEM,1000 DUTCH RIDGE RD,BEAVER,PA,15009,1000 Dutch Ridge Rd,Heritage Valley Health System,,Beaver,PA,15009,2021-12-10 07:27:43.202771
990723,NaN,540 E CANFIELD ST,DETROIT,MI,48201,540 E Canfield St,,,Detroit,MI,48201,2005-12-16 22:34:53.235684
990724,NaN,2115 RIVER FALLS DR,ROSWELL,GA,30076,2115 River Falls Dr,,,Roswell,GA,30076,2021-10-29 07:12:26.757082
990725,NaN,1 MEDICAL CENTER BLVD,WINSTON SALEM,NC,27157,1 Medical Center Blvd,,,Winston Salem,NC,27157,2018-11-29 20:46:30.661482


In [69]:
#Get polos
sql_query_polo = \
    F'''
    SELECT
    A.PARTY_ID,
    A.POST_CD_ID,
    A.CAT_CD_ID,
    A.FROM_DT
    FROM
    AMAEDW.PARTY_ADDR A
    WHERE
    A.THRU_DT IS NULL
    AND
    A.CAT_CD_ID <> 4159.0
    AND
    A.PARTY_ID IN {tuple(first_test.PARTY_ID)};
    '''
test_adds = pd.read_sql(con=AMAEDW, sql=sql_query_polo)
test_adds.head()

,PARTY_ID,POST_CD_ID,CAT_CD_ID,FROM_DT,PURPOSE_TYPE_ID
0,2459931,14415211,4213,2016-10-17 19:14:10.571734,415
1,2459931,15302125,4244,2016-10-17 15:16:33.375855,404
2,2459931,15608203,4338,2016-10-17 21:03:26.151786,403
3,2459931,19348495,4213,2016-10-19 06:50:48.261104,432
4,2459931,22572588,6455,2017-03-17 21:55:47.000000,467


In [72]:
#Get polos
sql_query_polo = \
    F'''
    SELECT
    A.PARTY_ID,
    A.POST_CD_ID,
    A.CAT_CD_ID,
    A.FROM_DT
    FROM
    AMAEDW.PARTY_ADDR A
    WHERE
    A.THRU_DT IS NULL
    AND
    A.CAT_CD_ID <> 4159.0;
    '''
all_adds = pd.read_sql(con=AMAEDW, sql=sql_query_polo)
all_adds.head()

,PARTY_ID,POST_CD_ID,CAT_CD_ID,FROM_DT
0,2472135,27643396,4213,2017-03-23 22:31:11
1,2472821,27640135,4213,2017-03-23 22:31:11
2,2474180,22857055,4213,2017-03-23 22:31:11
3,2476440,27152673,4213,2017-03-23 22:31:11
4,2477878,27642513,4213,2017-03-23 22:31:11


In [60]:
not_dea = test_adds[test_adds.CAT_CD_ID!=4159.0]

In [89]:
other_sources = pd.merge(dea_sourced, all_adds, on=['PARTY_ID','POST_CD_ID'])

In [76]:
21108/len(dea_sourced)

0.12533697523900006

In [78]:
19497/len(dea_sourced[dea_sourced.ME.isin(dpc.ME)])

0.12758898246853956

In [79]:
cat_cd = pd.read_csv('../../Data/DEA/cat_cd.csv')

In [88]:
all_adds = pd.merge(all_adds, cat_cd[['DESC','CAT_CD_ID']], on='CAT_CD_ID')

In [ ]:
#other source
#iqvia, symphony, dhc, datagov


In [93]:
len(other_sources)

25659

In [97]:
other_sources[other_sources.ME.isin(dpc.ME)].drop_duplicates(['ME','POST_CD_ID','DESC']).groupby('DESC').count()[['PARTY_ID']]

,PARTY_ID
DESC,
Account Management Customer Address,44
Account Management Customer Communication,1
Address Correction Service,102
External E-mail,32
FAST-TRACK,8
Group Roster/Census,14
HSG WEBPHYC DATA,8
Internal AMA E-mail,8
MEDEC,1


In [4]:
#Get polos
sql_query_polo = \
    F'''
    SELECT
    A.PARTY_ID,
    A.POST_CD_ID,
    A.CAT_CD_ID,
    A.FROM_DT
    FROM
    AMAEDW.PARTY_ADDR A
    WHERE
    A.THRU_DT IS NOT NULL
    AND
    A.CAT_CD_ID <> 4159.0;
    '''
null_adds = pd.read_sql(con=AMAEDW, sql=sql_query_polo)
null_adds.head()

,PARTY_ID,POST_CD_ID,CAT_CD_ID,FROM_DT
0,6162666,14443760,6455,2021-06-30 06:11:37
1,6163173,9142788,4213,2021-06-30 06:11:37
2,6163235,32264810,6455,2021-06-30 06:11:37
3,6163295,16938029,4213,2021-06-30 06:11:37
4,6163353,10102242,4213,2021-06-30 06:11:37


In [6]:
null_adds.to_csv('../../Data/DEA/enddated_edw.csv', index=False)

In [8]:
yup = pd.read_csv('../../Data/DEA/ONLY_DEA_EDW.csv')

In [12]:
BAH = pd.merge(yup, null_adds, on=['PARTY_ID','POST_CD_ID']).drop_duplicates('ME')

In [14]:
yup[~yup.PARTY_ID.isin(BAH.PARTY_ID)].to_csv('../../Data/DEA/THE_END.csv', index=False)

In [104]:
pd.merge(dea_sourced[~dea_sourced.ME.isin(other_sources.ME)&(dea_sourced.ME.isin(dpc.ME))], null_adds, on=['PARTY_ID','POST_CD_ID']).drop_duplicates(['PARTY_ID','POST_CD_ID'])

,PARTY_ID,ME,POST_CD_ID,CAT_CD_ID_x,ADDR_1,ADDR_2,ADDR_3,CITY,STATE,ZIP,FROM_DT_x,COMM_ID,PURPOSE_TYPE_DESC,CAT_CD_ID_y,FROM_DT_y
0,1889607,00301880456,26693275,4159.0,2081 Bronze Star Dr,,,Woodland,CA,95776,2016-10-19 23:20:07.135282,20738131,Office,6455,2016-10-29 19:08:41.606504
2,1906562,00301890419,28576274,4159.0,3700 W State Route 89A Ste B,None,None,Sedona,AZ,86336,2018-11-23 20:44:29.046568,22235817,Office,4213,2018-04-05 01:02:40.000000
3,1889654,00301930241,16517390,4159.0,7600 N La Cholla Blvd,,,Tucson,AZ,85741,2018-01-19 20:49:11.265029,8935392,Office,4213,2018-02-07 01:06:03.000000
4,1904721,00102861604,16482552,4159.0,334 Thomas More Pkwy,,,Crestview Hls,KY,41017,2018-08-17 20:46:11.092494,8383578,Office,4213,2018-11-09 03:30:13.000000
5,1904731,00102870867,16392850,4159.0,2055 E South Blvd,Ste 603,,Montgomery,AL,36116,2017-02-08 21:27:22.867175,6543702,Office,4213,2017-01-20 21:24:52.535746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60490,14054337,06001130020,29468468,4159.0,9321 W Thomas Rd Ste 205,None,None,Phoenix,AZ,85037,2021-12-10 07:29:23.995848,23214095,Office,6455,2019-09-21 11:48:42.000000
60491,14055435,04775160948,29906416,4159.0,6000 W Spring Creek Pkwy,Ste 200,,Plano,TX,75024,2021-10-22 07:12:05.862544,23610317,Office,6455,2020-09-30 04:04:13.000000
60492,14191718,03576170903,28930143,4159.0,5001 Hardy St,,,Hattiesburg,MS,39402,2019-08-09 21:12:02.350956,22481589,Office,4213,2019-08-07 00:28:51.000000
60493,14300558,00576141238,22003942,4159.0,526 S Tonopah Dr,Ste 160,,Las Vegas,NV,89106,2021-12-10 07:29:24.093878,15111201,Office,6455,2019-08-27 11:55:43.000000


In [106]:
dea_sourced[~dea_sourced.ME.isin(other_sources.ME)].to_csv('../../Data/DEA/DEA_Only.csv', index=False)

In [105]:
len(ppd)

1308519